# Análisis de Empleo en Ecuador: Explorando Datos a Nivel Provincial

## Un Enfoque Detallado en Samborondón: Tendencias Laborales y Desafíos Económicos - Enero de 2024

In [1]:
import os
import pandas as pd
from utils import add_isic, store_results, _get_city
LOCAL = True

https://unstats.un.org/unsd/classifications/Econ/Download/In%20Text/ISIC_Rev_4_english_structure.Txt


**Importación de bibliotecas**: En esta celda, se importan las bibliotecas necesarias para el procesamiento de datos. 
   - `os`: Se importa para permitir la interacción con el sistema operativo, lo que podría ser útil para manipular archivos y directorios.
   - `pandas as pd`: Pandas es una biblioteca de Python utilizada comúnmente para el análisis y manipulación de datos. Se importa bajo el alias "pd" para facilitar su referencia en el código.
   - `from utils import add_isic, store_results, _get_city`: S importa algunas funciones específicas desde un archivo llamado `utils.py`. Las funciones importadas son `add_isic`, `store_results` y `_get_city`.

In [2]:
def get_data(data_file):
    print('from: {}'.format(data_file), end='\t')
    data = pd.read_csv(
        data_file,
        usecols=['I01', 'I02', 'URP', 'TIPOACT', 'P29'])
    data = data.loc[(data.URP == 1) & (data.TIPOACT == " 1")]
    data = data.dropna()
    data.loc[:, 'cod'] = data.apply(lambda x: str(x['I01'])+'0'+str(x['I02']), axis=1)
    data.loc[:, 'count'] = 1
    return data

Esta celda contiene la función llamada `get_data()` que está diseñada para cargar y preprocesar datos desde un archivo CSV específico. Esta es la descripción para cada parte del código:

1. **Definición de la función `get_data(data_file)`**: Se define una función llamada `get_data` que toma un parámetro `data_file`, que es el nombre del archivo CSV que contiene los datos a procesar.

2. **Impresión de mensaje de estado**: Se imprime un mensaje indicando el origen del archivo de datos.

3. **Carga de datos CSV**: Se utiliza la función `pd.read_csv()` de la biblioteca Pandas para cargar el archivo CSV especificado en `data_file`. Se especifica el uso de solo las columnas `['I01' (Código geográfico 1), 'I02' (Código geográfico 2), 'URP' (Urbano/Rural), 'TIPOACT' (Tipo de actividad ), 'P29' (Actividad Económica)]` mediante el parámetro `usecols`.

4. **Filtrado de datos**: Se filtran los datos para incluir solo aquellas filas donde la columna 'URP' tiene el valor 1 y la columna 'TIPOACT' tiene el valor "1". Esto filtra solo datos urbanos de la población activa.

5. **Eliminación de filas con valores faltantes**: Se eliminan las filas que contienen valores faltantes (NaN) en cualquier columna.

6. **Creación de nuevas columnas**: Se crea una nueva columna llamada 'cod' que es una combinación de las columnas 'I01' e 'I02', que representa el código del cantón. También se crea una columna llamada 'count' que se establece en 1 para cada fila, representando cada puesto laboral.

7. **Retorno de los datos preprocesados**: Se devuelve el DataFrame resultante después de todas las transformaciones realizadas.

In [9]:
def parce_data(data, data_cod):
    for cod in data.cod.unique():
        print('search for: {}'.format(cod), end='\t')
        city_data = data.loc[data.cod==cod]
        try:
            city = cities.query('Valor=={}'.format(cod)).iloc[0, 1].strip()
        except:
            print("can't find city: {}".format(cod))
        else:
            if _get_city(city, '49', local=LOCAL)['count'] >= 1:
                city_data = city_data.loc[:, ['count', 'P29']].groupby('P29').sum().reset_index()
                print('found: {}'.format(city))
                exceptions = {'9999': '9820'}
                print('get_isic')
                city_data = add_isic(city_data, exceptions=exceptions, key='P29')
                city_data.columns = [
                    c.replace('count', '{}_nr_employed'.format(city.lower())) for c in city_data.columns]
                print('isic')
                store_results(city_data, city, data_cod,
                              isic_code=['P29_x', 'P29_y'],
                              country='ecuador', country_cod='49',
                              country_iso3='ECU', ilo_country_data='ECU-2019',
                              local=LOCAL)
            else:
                print('city: {} found but not in database'.format(cod))

Esta celda contiene una función llamada `parce_data(data, data_cod)` que está diseñada para procesar y analizar los datos preprocesados obtenidos de la función anterior. Esta es la descripción para cada parte del código:

8. **Definición de la función `parce_data(data, data_cod)`**: Se define una función llamada `parce_data` que toma dos parámetros: `data`, que es el DataFrame preprocesado obtenido de la función `get_data`, y `data_cod`, que es una contiene la tasa de empleo a nivel nacional.

10. **Iteración sobre los códigos únicos**: Se itera sobre los códigos únicos presentes en la columna 'cod' del DataFrame `data`. Estos códigos son identificadores asociados con ciudades o regiones del Ecuador.

11. **Impresión de mensaje de búsqueda**: Se imprime un mensaje indicando el código que se está buscando en los datos.

12. **Búsqueda de información de ciudad**: Se busca información de ciudad asociada con el código actual utilizando una consulta en un DataFrame llamado `cities`.

13. **Manejo de excepciones**: Se manejan las excepciones en caso de que no se pueda encontrar información de la ciudad para el código actual.

14. **Procesamiento de datos de ciudad encontrados**: Si se encuentra información de la ciudad, se realiza el procesamiento adicional de los datos asociados con esa ciudad.

15. **Procesamiento de datos de ciudad no encontrados**: Si la ciudad no está en la base de datos, se imprime un mensaje indicando que la ciudad no se encontró en la base de datos.

16. **Actualización y almacenamiento de resultados**: Se actualizan y almacenan los resultados procesados de la ciudad utilizando la función `store_results`.

In [10]:
cities = pd.read_excel('./cities.xlsx')
print('loaded cities data')

loaded cities data


Esta celda carga un archivo Excel llamado "cities.xlsx" en un DataFrame de pandas llamado "cities". Luego, imprime un mensaje indicando que los datos de las ciudades se han cargado correctamente. Aquí está la descripción para cada parte del código:

16. **Carga de datos de ciudades desde un archivo Excel**: Se utiliza la función `pd.read_excel()` de la biblioteca Pandas para cargar los datos del archivo Excel ubicado en el directorio actual con el nombre "cities.xlsx" en un DataFrame llamado "cities".

18. **Impresión de mensaje de estado**: Se imprime un mensaje indicando que los datos de las ciudades se han cargado correctamente. Esto proporciona una confirmación visual de que la carga de datos se realizó con éxito y los datos están listos para ser utilizados en el análisis.

In [11]:
print('loading national data...', end='\t')
data_file = './CPV2010M_CSV_Nacional/CPV2010M_Poblacion.csv'
data_cod = get_data(data_file)
data_cod = data_cod.loc[:, ['count', 'P29']].groupby('P29').sum().reset_index()

loading national data...	from: ./CPV2010M_CSV_Nacional/CPV2010M_Poblacion.csv	

Esta celda realiza dos acciones principales. Primero, imprime un mensaje indicando que se están cargando datos nacionales. Luego, llama a la función `get_data(data_file)` para procesar y obtener datos a partir de un archivo CSV específico y almacena el resultado en la variable `data_cod`. Finalmente, realiza una operación adicional en `data_cod`, seleccionando columnas específicas ('count' y 'P29'), agrupándolas por la columna 'P29' (Actividad Económica), sumando los valores de la columna 'count' y restableciendo el índice del DataFrame resultante.

Aquí está la descripción detallada para cada parte del código:

18. **Impresión de mensaje de estado**: Se imprime un mensaje indicando que se están cargando datos nacionales. Esto proporciona una confirmación visual de que se está llevando a cabo una tarea específica en el código.

20. **Carga de datos y procesamiento**: Se llama a la función `get_data(data_file)` con el nombre de archivo especificado en `data_file`. Esto procesa los datos del archivo y devuelve un DataFrame, que se almacena en la variable `data_cod`.

21. **Selección y agrupación de datos**: Selecciona las columnas 'count' y 'P29' del DataFrame `data_cod`, agrupa los datos por la columna 'P29', suma los valores de la columna 'count' para cada grupo y restablece el índice del DataFrame resultante. Esto parece ser una operación de agregación para obtener el recuento total de empleados por sector económico identificada por 'P29'.

In [12]:
exceptions = {'9999': '9820'}
data_cod = add_isic(data_cod, exceptions=exceptions, key='P29')
data_cod.columns = [c.replace('count', 'ecuador_nr_employed') for c in data_cod.columns]
print('OK')

OK


Esta celda realiza varias operaciones adicionales en el DataFrame `data_cod`. Aquí está la descripción para cada parte del código:

21. **Aplicación de excepciones ISIC**: Se define un diccionario llamado `exceptions` donde las claves representan códigos de excepción y los valores representan los códigos ISIC correspondientes. Luego, se llama a la función `add_isic()` para agregar códigos ISIC al DataFrame `data_cod` utilizando el diccionario de excepciones. Esta función asigna códigos ISIC a las categorías de empleo identificadas por el código 'P29'.

22. **Renombramiento de columnas**: Se renombran las columnas del DataFrame `data_cod` sustituyendo la parte 'count' de cada nombre de columna por 'ecuador_nr_employed'. Esto es una convención de nomenclatura para indicar que estas columnas representan el número de empleados en Ecuador.

24. **Impresión de mensaje de confirmación**: Se imprime un mensaje indicando que las operaciones se han realizado correctamente.

Estas operaciones adicionales estab preparando el DataFrame `data_cod` para su posterior almacenamiento o análisis, asignando códigos ISIC y renombrando las columnas para una mejor comprensión de los datos.



In [13]:
done = [
    'Bolivar',
    'Sucumbios',
    'ElOro',
    'Galapagos',
    'Azuay',
    'Nacional',
    'Tungurahua',
    'StoDomingo',
    'Imbabura',
    'ZonasNoDelim',
    'Manabi',
    'Cotopaxi',
    'Napo',
    'Morona',
    'Zamora',
    'Pastaza',
    'StaElena',
    'Pichincha',
    #'Guayas',
    'Canar',
    'LosRios',
    'Orellana',
    'Esmeraldas',
    'Carchi',
    'Loja',
    'Chimborazo'
]

24. Esta lista, llamada `done`, se utiliza para especificar qué provincias del Ecuador se procesarán en el siguiente comando. En este caso, solo los nombres que están comentados en la lista serán procesados. Por ejemplo, en la lista proporcionada, todas las provincias están comentadas excepto 'Guayas'.

El propósito de esta lista es controlar qué provincias del Ecuador se incluirán en el procesamiento de datos posterior. Al mantener todos los nombres de provincia comentados excepto uno, se garantiza que solo se procesen los datos para esa provincia específica (en este caso, 'Guayas').

Esto proporciona una forma conveniente de ajustar y controlar qué datos se procesan en un momento dado, permitiendo un enfoque selectivo en las provincias de interés sin la necesidad de cambiar manualmente el código fuente.

In [14]:
print('satart parsing data')
for data_folder in os.listdir(os.getcwd()):
    if os.path.isdir(data_folder) \
    and 'CPV2010M_CSV_' in data_folder \
    and 'Nacional' not in data_folder \
    and 'ZonasNoDelim' not in data_folder:
        for data_file in os.listdir(os.path.join(os.getcwd(), data_folder)):
            if 'Poblac' in data_file and data_file.split('_')[0] not in done:
                print(data_file)
                print('get data')
                data = get_data(os.path.join(data_folder, data_file))
                print('OK')
                parce_data(data, data_cod)

satart parsing data
Guayas_CSV_Poblac.csv
get data
from: CPV2010M_CSV_Guayas/Guayas_CSV_Poblac.csv	OK
search for: 901	found: Guayaquil
get_isic
isic
Empty DataFrame
Columns: [P29_x, P29_y, emp_circularity]
Index: []
search for: 902	found: Alfredo Baquerizo Moreno (Jujan)
get_isic
isic
Empty DataFrame
Columns: [P29_x, P29_y, emp_circularity]
Index: []
search for: 903	found: Balao
get_isic
isic
Empty DataFrame
Columns: [P29_x, P29_y, emp_circularity]
Index: []
search for: 904	found: Balzar
get_isic
isic
Empty DataFrame
Columns: [P29_x, P29_y, emp_circularity]
Index: []
search for: 905	found: Colimes
get_isic
isic
Empty DataFrame
Columns: [P29_x, P29_y, emp_circularity]
Index: []
search for: 906	found: Daule
get_isic
isic
Empty DataFrame
Columns: [P29_x, P29_y, emp_circularity]
Index: []
search for: 907	found: Durán
get_isic
isic
Empty DataFrame
Columns: [P29_x, P29_y, emp_circularity]
Index: []
search for: 908	found: El Empalme
get_isic
isic
Empty DataFrame
Columns: [P29_x, P29_y, emp_ci

Este fragmento de código es un bucle que itera sobre los directorios en el directorio de trabajo actual. Luego, para cada directorio, verifica algunas condiciones antes de procesar los archivos que se encuentran dentro de ese directorio.

Aquí está la descripción para cada parte del código:

25. **Impresión de mensaje de inicio**: Se imprime un mensaje indicando que se está comenzando el proceso de análisis de datos.

24. **Bucle sobre los directorios en el directorio actual**: Se utiliza un bucle `for` para iterar sobre los elementos devueltos por `os.listdir(os.getcwd())`, que son los nombres de los elementos en el directorio de trabajo actual.

27. **Verificación de condiciones**: Se verifica una serie de condiciones para determinar si se debe procesar el directorio actual. Estas condiciones son:
    - `os.path.isdir(data_folder)`: Verifica si el elemento actual es un directorio.
    - `'CPV2010M_CSV_' in data_folder`: Verifica si el nombre del directorio contiene la subcadena 'CPV2010M_CSV_'.
    - `'Nacional' not in data_folder`: Verifica si la subcadena 'Nacional' no está presente en el nombre del directorio.
    - `'ZonasNoDelim' not in data_folder`: Verifica si la subcadena 'ZonasNoDelim' no está presente en el nombre del directorio.

28. **Bucle sobre los archivos en el directorio actual**: Si se cumplen todas las condiciones anteriores, se utiliza otro bucle `for` para iterar sobre los archivos dentro del directorio actual. 

29. **Verificación de archivos específicos**: Para cada archivo en el directorio actual, se verifica si contiene la subcadena 'Poblac' en su nombre y si el primer elemento del nombre del archivo (obtenido dividiendo el nombre por '_') no está en la lista `done` (es decir, no está comentado). Si estas condiciones se cumplen, se imprime el nombre del archivo y se llama a la función `get_data()` para procesar los datos del archivo.

30. **Impresión de mensajes de estado**: Se imprime un mensaje indicando que se está obteniendo los datos del archivo.

31. **Fin de la iteración**: El código finaliza con un comentario que indica que la función `parce_data()` se llamará con los datos obtenidos.

